In [ ]:
import requests

In [41]:
from bs4 import BeautifulSoup

In [ ]:
import pandas as pd

# Задание 1

In [39]:
KEYWORDS = ['python', 'парсинг', 'iOS', 'разработка']

In [43]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [44]:
# извлекаем посты

posts = soup.find_all('article', class_='post')
# posts

In [45]:

habr_articles = pd.DataFrame()

for post in posts:
    post_id = post.parent.attrs.get('id')
    
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
        
  # находим анонс текста статьи  
    texts = post.find_all('div', class_='post__body post__body_crop')

    
    for word in texts:
            text_lower = word.text.lower()

           # ищем вхождение хотя бы одного желаемого слова
        
                          
            for desired in KEYWORDS:
                if desired in text_lower:
               # пост нам интересен 
               
                    title_element = post.find('a', class_='post__title_link')
                    
                    date = post.find('span', class_='post__time')
                   
                    row = {'date': date.text, 'title': title_element.text, 'link': title_element.attrs.get('href')}
                    
                    habr_articles = pd.concat([habr_articles, pd.DataFrame([row])])  
                    

            
habr_articles

""


# Дополнительная часть

In [46]:
# добавляем извлечение хабов из постов, чтобы отбирать только нужные

habr_articles = pd.DataFrame()

for post in posts:
    post_id = post.parent.attrs.get('id')
    
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
        


    texts = post.find_all('div', class_='post__body post__body_crop')

    
    for word in texts:
            text_lower = word.text.lower()
           # print(text_lower)
           
            
            title_element = post.find('a', class_='post__title_link')
            link = title_element.attrs.get('href')
            
            # открываем саму статью
            link_req = requests.get(link)
            link_soup = BeautifulSoup(link_req.text, 'html.parser')
         
        
            link_text = link_soup.find_all('div', class_='post__text post__text_v2')

           
            
            for word in link_text:
                text_lower = word.text.lower()
               # ищем вхождение хотя бы одного желаемого слова      
                          
                for desired in KEYWORDS:
                    if desired in text_lower:
                       # пост нам интересен 
               
                        title_element = post.find('a', class_='post__title_link')
                    
                        date = post.find('span', class_='post__time')
                   
                        row = {'date': date.text, 'title': title_element.text, 'link': title_element.attrs.get('href'), 'text': text_lower}
                    
                        if title_element.text not in habr_articles.values:
                            habr_articles = pd.concat([habr_articles, pd.DataFrame([row])])  
                    
            
habr_articles

<ipython-input-46-2fa955520a00>:48: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if title_element.text not in habr_articles.values:


,date,title,link,text
0,сегодня в 16:05,Трудности аутстаффинга: борьба с культурными р...,https://habr.com/ru/post/532974/,как избежать недопонимания между сотрудниками ...
0,сегодня в 14:26,Real Time API в контексте Apache Kafka,https://habr.com/ru/company/otus/blog/532954/,всем привет. уже в декабре в otus стартует нов...
0,сегодня в 14:03,Быстрый старт: гайд по автоматизированному тес...,https://habr.com/ru/company/redmadrobot/blog/5...,"привет! меня зовут сергей иванов, я ведущий ра..."


# Задание 2

In [1]:
import pandas as pd

In [2]:
import requests

In [3]:
import json

In [4]:
import time

In [35]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [36]:
EMAIL = ['telukhlarisa@gmail.com', 'telukhlarisa@mail.ru']

In [37]:
headers = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast'
}

In [38]:
data_list = pd.DataFrame()

for email_address in EMAIL:
    req = requests.post(URL, json = {'emailAddresses':[email_address]}, headers = headers)

    data = json.loads(req.text)


    for item, value in data.items(): 
        if item == 'breaches':
            for i in value.values():
                row = {'email': email_address, 'date': i['publishDate'], 'site': i['site'], 'text': i['description']}
                data_list = pd.concat([data_list, pd.DataFrame([row])]) 
        
    time.sleep(2)

data_list

,email,date,site,text
0,telukhlarisa@gmail.com,2018-11-21T00:00:00Z,animoto.com,"In July 2018, the online video site Animoto su..."
0,telukhlarisa@gmail.com,2016-10-21T00:00:00Z,linkedin.com,"In 2012, online professional networking platfo..."
0,telukhlarisa@gmail.com,2017-04-10T00:00:00Z,google.com/gmail,This collection of compromised Gmail users con...
0,telukhlarisa@gmail.com,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,telukhlarisa@gmail.com,2019-12-13T00:00:00Z,peopledatalabs.com,"In October 2019, a security researcher discove..."
0,telukhlarisa@mail.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,telukhlarisa@mail.ru,2018-12-13T00:00:00Z,houzz.com,"In July 2018, the housing site Houzz was alleg..."
0,telukhlarisa@mail.ru,2016-10-25T00:00:00Z,last.fm,"In March 2012, the British music streaming ser..."
